In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load the Pascal VOC 2007 dataset from TensorFlow Datasets
# This dataset includes bounding boxes and labels for objects
dataset, info = tfds.load('voc/2007', split='train', with_info=True)

# Dataset features: image, objects (bounding boxes, labels)
print(info.features)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
IMAGE_SIZE = 416

# Helper function to preprocess images and labels
def preprocess_data(sample):
    image = sample['image']
    bbox = sample['objects']['bbox']  # Bounding box coordinates
    label = sample['objects']['label']  # Object labels

    # Resize the image to YOLOv3 input size
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

    return image, bbox, label

# Apply preprocessing to the dataset
dataset = dataset.map(preprocess_data)


In [ ]:
from tensorflow.keras.models import load_model

# Load a pre-trained YOLOv3 model (you can download YOLOv3 weights and architecture)
# Replace 'yolov3.h5' with the path to your downloaded weights in Keras format
yolov3_model = load_model('yolov3.h5')  # Ensure you have the right weights


In [ ]:
yolov3_model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss='binary_crossentropy',  # YOLOv3 uses a custom loss, adapt accordingly if needed
    metrics=['accuracy']
)


In [ ]:
BATCH_SIZE = 32
EPOCHS = 50

# Create a batched dataset
dataset = dataset.batch(BATCH_SIZE)

# Fine-tune the YOLOv3 model on Pascal VOC dataset
history = yolov3_model.fit(
    dataset,
    epochs=EPOCHS,
    verbose=1
)


In [ ]:
# Take a sample image from the test dataset for prediction
sample_image, sample_bbox, sample_label = next(iter(dataset))

# Make predictions using the YOLOv3 model
yolo_predictions = yolov3_model.predict(tf.expand_dims(sample_image, axis=0))

# Post-process the predictions (filtering low-confidence boxes and non-max suppression)
# Note: YOLOv3's raw outputs need to be processed into bounding boxes, class scores, and labels
def postprocess_predictions(predictions, confidence_threshold=0.5, iou_threshold=0.4):
    # Implement logic to convert raw YOLOv3 outputs into human-readable bounding boxes and labels
    pass

# Example: Display the predictions on the image
plt.imshow(sample_image.numpy())
plt.show()


In [ ]:
# Assuming we have true labels and predictions
from sklearn.metrics import precision_recall_curve, average_precision_score

# Precision-Recall curve
precision, recall, _ = precision_recall_curve(true_labels, yolo_predictions)

# Calculate Mean Average Precision (mAP)
map_score = average_precision_score(true_labels, yolo_predictions)

print(f"mAP Score: {map_score}")
